In [ ]:
Ex_Ruten(fail)
網頁好像有微調+看不懂

In [ ]:
露天拍賣 爬蟲要點
1. 在收集產品清單時需要透過 F12 的檢查功能來找出 API
2. 產品描述要呼叫另外一個 API 來取資料
3. API中無page參數，換頁要用offset參數(記錄要從哪裡開始加載更多資料)<br>
    由於每次查詢是80筆(limit參數)，因此每次查詢的 offset 是用 1 + 80*第幾次查詢來抓資料

In [9]:
import re
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup


#在這裡沒有設定 User-Agent 會被拒絕連線，因此當我們無法成功取回資料時的第一個步驟一定是加上 User-Agent 來進行測試!
headers = {'User-Agent':'GoogleBot'}

# 商品分類
cateid = '001100020027'
pages = 2

prodids = []

for page in list(range(1, pages)):
    offset = 1 + 80*page
    url = 'https://rtapi.ruten.com.tw/api/search/v3/index.php/core/prod?cateid={}&sort=rnk%2Fdc&offset={}&limit=80&2653512&_callback=jsonpcb_CoreProd'
    url = url.format(cateid, offset)
    resp = requests.get(url, headers=headers)
    #用正則把前後多餘的部分取代掉，只留{}中字典的部分
    data = re.sub(r'try\{jsonpcb_CoreProd\(|\);\}catch\(e\)\{if\(window.console\)\{console.log\(e\);\}\}','', resp.text)
    for prod in json.loads(data)['Rows']:
        prodids.append(prod['Id'])
    prodids = list(set(prodids))  #取得id list
    print('There are {} prods in list.'.format(len(prodids)))

There are 80 prods in list.


In [2]:
prodids

['30202682547515',
 '30202190923135',
 '22024668435945',
 '30202744775630',
 '30203086999021',
 '30202725785641',
 '30202634139250',
 '22021594432802',
 '30201118288688',
 '30202642013104',
 '21901425730161',
 '30202629237174',
 '30203083281684',
 '30203007941278',
 '30202689841508',
 '30202641506649',
 '30203086284512',
 '30202521303528',
 '22028855845054',
 '30194449037071',
 '30203033954462',
 '30202700021388',
 '30203071976907',
 '30202677460760',
 '30194602319312',
 '21935595313118',
 '30203023909304',
 '30203062350385',
 '30202725305065',
 '22028852582329',
 '22029910610934',
 '30202733286466',
 '30202730208018',
 '30202699876035',
 '21939003559095',
 '30203068185668',
 '30203007625299',
 '22021578462290',
 '30202278039107',
 '30203084095352',
 '30202134602722',
 '30202636368694',
 '30203059032625',
 '30203040062437',
 '30203053745754',
 '30203074892592',
 '30203047346361',
 '30203070047512',
 '30202646072541',
 '30203080039042',
 '30203021393039',
 '30203012314100',
 '3020302336

In [ ]:
爬取產品資料
這裡在抓產品名稱、幣別、價格、商品分類等等資訊的方法相當單純，就是簡單 request 產品的網址就可以了! 
而產品的網址就是在前面「收集產品清單」階段中收集到的產品 ID前面加上'https://goods.ruten.com.tw/item/show?'的網址即可。
例如ID：30201451592585 產品的網址就是 https://goods.ruten.com.tw/item/show?30201451592585

比較複雜的地方在於產品描述不容易獲取，在這裡有兩個反爬蟲的機制：
request 的網址藏在前面回傳的 descriptionUrl 中
這裡要花時間找一下產品描述的 API 網址，結果是藏在前面 request 的結果當中。

Referer：要在 request 的 headers 加上 referer，這個參數是告訴對方的伺服器說我們是從哪裡進來到這個API的。

In [3]:
df = []
for i, prodid in enumerate(prodids):
    print('='*60)  #60個'='
    print('Dealing with {}: {}'.format(i, prodid))
    headers = {'User-Agent':'GoogleBot'}
    url = 'https://goods.ruten.com.tw/item/show?' + prodid
    resp = requests.get(url, headers=headers)
    soup = BeautifulSoup(resp.text)
    data1 = json.loads(soup.findAll('script',{'type':'application/ld+json'})[0].text)
    data2 = json.loads(soup.findAll('script',{'type':'application/ld+json'})[1].text)
    data3 = json.loads(soup.findAll('script',{'type':'application/ld+json'})[2].text)
    # 產品名稱
    print(data1['name'])
    # 幣別
    print(data1['offers']['priceCurrency'])
    # 價格
    print(data1['offers']['price'])
    # 商品分類
    print(r'/'.join([i['item']['name'] for i in data3['itemListElement']]))
    # 商品描述
    descurl = re.findall(r'goods_comments.php\?id=.*?o=[0-9]{1,}',resp.text)[0]
    descurl = re.sub('&amp;','&',descurl)
    descurl = 'https://goods.ruten.com.tw/item/' + descurl
    headers = {'User-Agent':'GoogleBot',
               'Referer': 'https://goods.ruten.com.tw/item/show?{}'.format(prodid)}
    resp2 = requests.get(descurl, headers=headers)
    soup2 = BeautifulSoup(resp2.text)
    desc = ' '.join(i.text for i in soup2.findAll('p'))
    
    ndf = pd.DataFrame([{'name': data1['name'],
                         'description':data1['description'],
                         'productId':data1['productId'],
                         'brand_name':data1['brand']['name'],
                         'priceCurrency':data1['offers']['priceCurrency'],
                         'price':data1['offers']['price'],
                         'contentLocation':data2['contentLocation'],
                         'item_name':r'/'.join([i['item']['name'] for i in data3['itemListElement']]),
                         'describe':desc}])
    df.append(ndf)

df = pd.concat(df, ignore_index=True)

df.info()

Dealing with 0: 30202682547515


IndexError: list index out of range

In [4]:
resp.text

'<!DOCTYPE html>\n<html xmlns:og="http://ogp.me/ns#" xmlns:fb="http://www.facebook.com/2008/fbml" xml:lang="zh-TW" lang="zh-TW">\n<head>\n  <meta charset="utf-8">\n  <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n    <title>Panasonic Toughbook CF-19 i5 8GB 500GB Chart Plotter Marine Navigation GPS Touch - PChomeUSA 海外代購</title>\n\n    <meta name="description" content="Panasonic Toughbook CF-19 i5 8GB 500GB Chart Plotter Marine Navigation GPS Touch" />\n    <meta itemprop="name" content="Panasonic Toughbook CF-19 i5 8GB 500GB Chart Plotter Marine Navigation GPS Touch - 露天拍賣">\n    <meta itemprop="description" content="Panasonic Toughbook CF-19 i5 8GB 500GB Chart Plotter Marine Navigation GPS Touch">\n    <meta itemprop="image" content="https://img.ruten.com.tw/s9/ebay/aa6/9c1/maxbgwm2%40ebay/4/6d/3b/20512830cacab0ff4223d9e471d0066f_30202682547515.jpg">\n    <meta property="og:title" content="Panasonic Toughbook CF-19 i5 8GB 500GB Chart Plotter Marine Navigation GPS Touc

In [ ]:
df.to_excel('./RuTen.xlsx')